In [0]:
%sql

CREATE SCHEMA IF NOT EXISTS youtube_project.bronze;

In [0]:
%sql

CREATE SCHEMA IF NOT EXISTS youtube_project.silver;

In [0]:
%sql

CREATE SCHEMA IF NOT EXISTS youtube_project.gold;

In [0]:
df = spark.read.format('parquet')\
            .option('inferSchema',True)\
            .load('abfss://bronze@saprojectyt.dfs.core.windows.net/')

In [0]:
%sql
SELECT * FROM PARQUET.`abfss://bronze@saprojectyt.dfs.core.windows.net/`

In [0]:
df.display()

In [0]:
df.printSchema()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

df = df.withColumn('model_category',split(col('model_id'),'-')[0])
df.display()

In [0]:
df = df.withColumn('Revenue_perUnit',col('Revenue')/col('Units_Sold'))
display(df.select(col('Units_Sold'),col('Revenue'),col('Revenue_perUnit')).display())

In [0]:
df_unitSolds = df.groupBy([col('Year'), col('BranchName')]).agg(sum('Units_Sold').alias('ToatlUnit_Sold'))
df_unitSolds.sort(col('Year'),col('ToatlUnit_Sold'),ascending = [1,0])

In [0]:
df.write.format('parquet')\
        .mode('overwrite')\
        .option('path','abfss://silver@saprojectyt.dfs.core.windows.net/carsales/sales.parquet')\
        .save()


In [0]:
%sql

CREATE TABLE IF NOT EXISTS youtube_project.silver.sales
(
  Branch_ID STRING,
  Dealer_ID string,  
  Model_ID string ,
  Revenue long ,
  Units_Sold long ,
  Date_ID string,
  Day integer,
  Month integer ,
  Year integer,
  BranchName string,
  DealerName string,
  Model_Category STRING
) LOCATION 'abfss://silver@saprojectyt.dfs.core.windows.net/sales'

In [0]:
%sql
INSERT INTO youtube_project.silver.sales